In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import trange
from scipy.stats import wilcoxon
import ants

from metrics import *

metrics = Metrics()

import pdb

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-mcwg2cyd because the default path (/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [17]:
# Compute average surface distance
data = 'lits'
models = ['ablation-1', 'ablation-2', 'ablation-3', 'ablation-4']

for model in models:
    paths = pd.read_csv('/rsrch1/ip/aecelaya/github/MIST/mist/path_csv/lits_paths.csv'.format(data))
    ids = list(paths['id'])

    truth = list()
    preds_pocket = list()
    preds_full = list()
    
    preds_pocket_clean = list()
    preds_full_clean = list()
    for i in ids:
        id_num = i.split('_')[-1]
        truth.append('/rsrch1/ip/aecelaya/data/{}/raw/train/{}/segmentation_binary-{}.nii.gz'.format(data, i, id_num))
        preds_pocket.append('/rsrch1/ip/aecelaya/data/{}/predictions/pocketnet/ablation/{}/{}.nii.gz'.format(data, model, i))
        #preds_full.append('/rsrch1/ip/aecelaya/data/{}/predictions/pocketnet/{}_{}/{}.nii.gz'.format(data, model, 'full', i))
        
        preds_pocket_clean.append('/rsrch1/ip/aecelaya/data/{}/predictions/pocketnet/ablation/{}/{}_clean.nii.gz'.format(data, model, i))
        #preds_full_clean.append('/rsrch1/ip/aecelaya/data/{}/predictions/pocketnet/{}_{}/{}_clean.nii.gz'.format(data, model, 'full', i))

    avg_surfs_pocket = list()
    avg_surfs_full = list()
    for i in trange(len(ids)):
        pocket_pred = ants.image_read(preds_pocket[i])
        pocket_pred = ants.get_mask(pocket_pred, cleanup = 2)
        ants.image_write(pocket_pred, preds_pocket_clean[i])

        # full_pred = ants.image_read(preds_full[i])
        # full_pred = ants.get_mask(full_pred, cleanup = 2)
        # ants.image_write(full_pred, preds_full_clean[i])
            
        avg_surfs_pocket.append(metrics.dice_sitk(truth[i], preds_pocket_clean[i]))
        #avg_surfs_full.append(metrics.dice_sitk(truth[i], preds_full_clean[i]))


    # print('===================== Full =====================')
    # print('Mean = {}'.format(np.round_(np.nanmean(avg_surfs_full), 4)))
    # print('Std = {}'.format(np.round_(np.nanstd(avg_surfs_full), 4)))
    # print('Median = {}'.format(np.round_(np.nanmedian(avg_surfs_full), 4)))
    print('===================== Pocket =====================')
    print('Mean = {}'.format(np.round_(np.nanmean(avg_surfs_pocket), 4)))
    print('Std = {}'.format(np.round_(np.nanstd(avg_surfs_pocket), 4)))
    print('Median = {}'.format(np.round_(np.nanmedian(avg_surfs_pocket), 4)))
    # print('===================== p-value =====================')
    # print('p-value = {}'.format(wilcoxon(avg_surfs_full, avg_surfs_pocket)))

  0%|          | 0/131 [00:00<?, ?it/s]

===================== Pocket =====================
Mean = 0.922
Std = 0.1053
Median = 0.9506


  0%|          | 0/131 [00:00<?, ?it/s]

===================== Pocket =====================
Mean = 0.907
Std = 0.145
Median = 0.9503


  0%|          | 0/131 [00:00<?, ?it/s]

===================== Pocket =====================
Mean = 0.9234
Std = 0.0957
Median = 0.9518


100%|██████████| 131/131 [3:09:35<00:00, 86.83s/it]  

===================== Pocket =====================
Mean = 0.9213
Std = 0.109
Median = 0.9526


In [17]:
data = 'nfbs'
model = 'unet'
metric = 'BRAIN_dice'

full = pd.read_csv('/tf/github/MIST/mist/results/{}_full_{}_results.csv'.format(data, model))
pocket = pd.read_csv('/tf/github/MIST/mist/results/{}_pocket_{}_results.csv'.format(data, model))

wilcoxon(full.iloc[0:-5]['{}'.format(metric)], pocket.iloc[0:-5]['{}'.format(metric)])

WilcoxonResult(statistic=0.0, pvalue=5.960464477539063e-08)

In [8]:
pocket

,id,BRAIN_dice,BRAIN_haus95,BRAIN_avg_surf
0,A00060407,0.987822,0.0,0.657803
1,A00037848,0.987374,0.0,0.420173
2,A00058952,0.989289,0.0,0.444770
3,A00033747,0.991061,0.0,0.275176
4,A00060430,0.988956,0.0,0.290659
5,A00040628,0.990830,0.0,0.252788
6,A00059845,0.989824,0.0,0.290815
7,A00028185,0.984602,0.0,0.616104
8,A00037511,0.989882,0.0,0.575424
9,A00057372,0.986474,0.0,0.565599
